In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
def get_sites(string, site_type='ACT_SITE'):
    """string=site entry
    Returns the sites as a list, e.g. if 
    string = 'ACT_SITE 85;  /evidence=...; ACT_SITE 110;  /evidence=...'
    returns [85, 110]"""
    
    pattern = re.compile(f'{site_type} [0-9]+;')
    matches = pattern.findall(string)
    get_site = lambda match: int(match.split(' ')[1][:-1])
    return [get_site(match) for match in matches]
get_binding_sites = lambda string: get_sites(string, 'BINDING')

SyntaxError: invalid syntax (<ipython-input-2-8e267fa9e94d>, line 7)

In [113]:
df = pd.read_csv('uniprot.tab', sep='\t', lineterminator='\n')
df = df.fillna('')

In [114]:
df[:5]

,Entry,Entry name,Status,Protein names,Gene names,Organism,Length,Sequence,Active site,Binding site
0,Q9SJT1,SAE2_ARATH,reviewed,SUMO-activating enzyme subunit 2 (EC 2.3.2.-) ...,SAE2 EMB2764 At2g21470 F3K23.23,Arabidopsis thaliana (Mouse-ear cress),700,MATQQQQSAIKGAKVLMVGAGGIGCELLKTLALSGFEDIHIIDMDT...,"ACT_SITE 168; /note=""Glycyl thioester interme...","BINDING 43; /note=""ATP""; /evidence=""ECO:0000..."
1,P56221,SCYD_MAGO7,reviewed,Scytalone dehydratase (SD) (SDH) (EC 4.2.1.94),SDH1 MGG_05059,Magnaporthe oryzae (strain 70-15 / ATCC MYA-46...,172,MGSQVQKSDEITFSDYLGLMTCVYEWADSYDSKDWDRLRKVIAPTL...,"ACT_SITE 85; /evidence=""ECO:0000305|PubMed:78...","BINDING 30; /note=""Substrate""; /evidence=""EC..."
2,Q0VCA5,SAMH1_BOVIN,reviewed,Deoxynucleoside triphosphate triphosphohydrola...,SAMHD1,Bos taurus (Bovine),589,MQSADSQNTPKRPRRDGSPRTPPDSPLADAETSPSHDLDPDYRTWG...,"ACT_SITE 221; /evidence=""ECO:0000250|UniProtK...","BINDING 104; /note=""GTP""; /evidence=""ECO:000..."
3,O45539,SRC2_CAEEL,reviewed,Tyrosine protein-kinase src-2 (EC 2.7.10.2) (S...,src-2 kin-22 F49B2.5,Caenorhabditis elegans,507,MGSCIGKEDPPPGATSPVHTSSTLGRESLPSHPRIPSIGPIAASSS...,"ACT_SITE 358; /note=""Proton acceptor""; /evid...","BINDING 268; /note=""ATP""; /evidence=""ECO:000..."
4,Q27324,RYK1_DROME,reviewed,Tyrosine-protein kinase Drl (EC 2.7.10.1) (Pro...,drl lio CG17348,Drosophila melanogaster (Fruit fly),610,MAPNLLTIGLLLTLIASGQAHLNIFLNLHEVLRLIGVSAELYYVRE...,"ACT_SITE 468; /note=""Proton acceptor""; /evid...","BINDING 371; /note=""ATP""; /evidence=""ECO:000..."


In [115]:
df.loc[:, 'Active site'] = df.loc[:, 'Active site'].apply(get_sites)
df.loc[:, 'Binding site'] = df.loc[:, 'Binding site'].apply(get_binding_sites)

In [116]:
df[:5]

,Entry,Entry name,Status,Protein names,Gene names,Organism,Length,Sequence,Active site,Binding site
0,Q9SJT1,SAE2_ARATH,reviewed,SUMO-activating enzyme subunit 2 (EC 2.3.2.-) ...,SAE2 EMB2764 At2g21470 F3K23.23,Arabidopsis thaliana (Mouse-ear cress),700,MATQQQQSAIKGAKVLMVGAGGIGCELLKTLALSGFEDIHIIDMDT...,[168],"[43, 67]"
1,P56221,SCYD_MAGO7,reviewed,Scytalone dehydratase (SD) (SDH) (EC 4.2.1.94),SDH1 MGG_05059,Magnaporthe oryzae (strain 70-15 / ATCC MYA-46...,172,MGSQVQKSDEITFSDYLGLMTCVYEWADSYDSKDWDRLRKVIAPTL...,"[85, 110]","[30, 50, 53, 131]"
2,Q0VCA5,SAMH1_BOVIN,reviewed,Deoxynucleoside triphosphate triphosphohydrola...,SAMHD1,Bos taurus (Bovine),589,MQSADSQNTPKRPRRDGSPRTPPDSPLADAETSPSHDLDPDYRTWG...,[221],"[104, 107, 137, 152, 198, 304, 308, 322, 347, ..."
3,O45539,SRC2_CAEEL,reviewed,Tyrosine protein-kinase src-2 (EC 2.7.10.2) (S...,src-2 kin-22 F49B2.5,Caenorhabditis elegans,507,MGSCIGKEDPPPGATSPVHTSSTLGRESLPSHPRIPSIGPIAASSS...,[358],[268]
4,Q27324,RYK1_DROME,reviewed,Tyrosine-protein kinase Drl (EC 2.7.10.1) (Pro...,drl lio CG17348,Drosophila melanogaster (Fruit fly),610,MAPNLLTIGLLLTLIASGQAHLNIFLNLHEVLRLIGVSAELYYVRE...,[468],[371]
